# Case 0: Step-by-Step Visualization Example

This notebook demonstrates a simplified step-by-step visualization of the polymer reaction process:

1. Build monomer and merge into polymer
2. Add second monomer with move/rotate, merge into polymer
3. Connect monomers and remove water

Each step exports a LAMMPS data file for visualization.

**Key features:**
- Manual positioning of monomers using move/rotate operations
- Step-by-step reaction visualization
- LAMMPS export at each stage

## Step 1: Import Required Libraries

Import all necessary modules from MolPy for building monomers, handling reactions, positioning molecules, and exporting to LAMMPS format.

In [1]:
import numpy as np
import molpy as mp
from molpy.core.atomistic import Atom, Atomistic
from molpy.core.entity import Entity
from molpy.external import RDKitAdapter, Generate3D
from molpy.parser.smiles import parse_bigsmiles, bigsmilesir_to_monomer
from molpy.reacter import (
    Reacter,
    select_hydroxyl_group,
    form_single_bond,
)
from molpy.reacter.selectors import select_port_atom
from molpy.reacter.utils import find_neighbors
from molpy.io.data.lammps import LammpsDataWriter
from molpy.typifier.atomistic import OplsAtomisticTypifier
from pathlib import Path

## Step 2: Load Force Field

Load the OPLS-AA force field and create a typifier. We use `strict_typing=False` to handle novel structures that may arise from reactions.

In [2]:
# Load force field
ff = mp.io.read_xml_forcefield("oplsaa.xml")
typifier = OplsAtomisticTypifier(ff, strict_typing=True)

print("✅ Force field loaded successfully")

/workspaces/molcrafts/molpy/src/molpy/potential/dihedral/opls.py:91: UserWarning: RB coefficients do not lie on the ideal 4-term OPLS manifold (C0+C1+C2+C3+C4 = 10.041600, expected ≈ 0). Conversion will preserve forces and relative energies exactly, but will introduce a constant energy offset of ΔE = 10.041600 kJ/mol. This does not affect MD simulations.
  warnings.warn(


✅ Force field loaded successfully


## Step 3: Define Helper Functions

Define helper functions for:
- **Reaction selectors**: Identify reaction sites and leaving groups
- **Monomer builder**: Parse BigSMILES and generate 3D coordinates
- **LAMMPS exporter**: Export structures to LAMMPS format

In [3]:
def select_carbon_from_oh(assembly: Atomistic, port_name: str) -> Entity:
    """Select the carbon atom connected to -OH oxygen (port)."""
    port_o = select_port_atom(assembly, port_name)
    c_neighbors = find_neighbors(assembly, port_o, element="C")
    return c_neighbors[0]


def select_h_from_oh(assembly: Atomistic, port_atom: Entity) -> list[Entity]:
    """Select H from -OH group when port_atom is the O atom."""
    h_neighbors = find_neighbors(assembly, port_atom, element="H")
    return [h_neighbors[0]]


def build_monomer() -> Atomistic:
    """Build monomer with -OH end groups from BigSMILES."""
    bigsmiles = '{[<]OCCOCCOCCOCCO[>]}'
    ir = parse_bigsmiles(bigsmiles)
    monomer = bigsmilesir_to_monomer(ir)
    
    adapter = RDKitAdapter(internal=monomer)
    generate_3d = Generate3D(add_hydrogens=True, embed=True, optimize=True, update_internal=True)
    adapter = generate_3d(adapter)
    monomer = adapter.get_internal()
    
    
    return monomer


def export_frame_to_lammps(atomistic: Atomistic, output_path: Path, box_size: float = 20.0) -> None:
    """Export Atomistic to LAMMPS after typification."""
    frame = atomistic.to_frame()
    frame.metadata["box"] = mp.Box.cubic(length=box_size)
    
    n_atoms = frame["atoms"].nrows
    
    # Ensure ID field
    if "id" not in frame["atoms"]:
        ids = [atom.get("id", i + 1) for i, atom in enumerate(atomistic.atoms)]
        frame["atoms"]["id"] = np.array(ids, dtype=int)
    
    # Ensure mol field
    frame["atoms"]["mol"] = np.array([1] * n_atoms, dtype=int)
    
    # Ensure charge field is available as 'q' (LAMMPS uses 'q')
    if "charge" in frame["atoms"]:
        frame["atoms"]["q"] = frame["atoms"]["charge"]
    
    writer = LammpsDataWriter(output_path, atom_style="full")
    writer.write(frame)

print("✅ Helper functions defined")

✅ Helper functions defined


## Step 4: Build First Monomer

Build the first monomer from BigSMILES notation, generate topology, type atoms, and merge into a polymer structure. Export the result to LAMMPS format.

In [4]:
# Create output directory
output_dir = Path("case0_output")
output_dir.mkdir(parents=True, exist_ok=True)

# Build monomer
monomer = build_monomer()

# Generate topology
monomer.get_topo(gen_angle=True, gen_dihe=True)

# Type the monomer
typifier.typify(monomer)

# Create polymer and merge monomer
polymer = Atomistic()
polymer.merge(monomer)

print(f"✅ Step 1 completed:")
print(f"   Monomer: {len(monomer.atoms)} atoms")
print(f"   Polymer: {len(polymer.atoms)} atoms")

# Export to LAMMPS
export_frame_to_lammps(polymer, output_dir / "step1_monomer.data", box_size=20.0)
print(f"   Exported: {output_dir / 'step1_monomer.data'}")

# Keep reference to left monomer for later steps
left_monomer = monomer

✅ Step 1 completed:
   Monomer: 31 atoms
   Polymer: 31 atoms
   Exported: case0_output/step1_monomer.data


## Step 5: Add Second Monomer with Positioning

Add a second monomer to the polymer by:
1. Creating a copy of the monomer
2. Assigning new atom IDs
3. Finding port atoms ("out" on polymer, "in" on new monomer)
4. Calculating translation and rotation to position the monomer
5. Merging into polymer and re-typing

In [5]:
# Create second monomer copy
monomer2 = build_monomer()

# Assign new IDs
max_id = max((atom.get("id", 0) for atom in polymer.atoms), default=0)
for atom in monomer2.atoms:
    max_id += 1
    atom["id"] = max_id

# Get port atoms
port1_atom = None  # "out" port of polymer
port2_atom = None  # "in" port of monomer2

for atom in polymer.atoms:
    if atom.get("port") == ">":
        port1_atom = atom
        break

for atom in monomer2.atoms:
    if atom.get("port") == "<":
        port2_atom = atom
        break

# Calculate translation: move monomer2 to align ports (head-to-head)
port1_pos = np.array([port1_atom["x"], port1_atom["y"], port1_atom["z"]])
port2_pos = np.array([port2_atom["x"], port2_atom["y"], port2_atom["z"]])

# Calculate direction from polymer center to port1
polymer_coords = np.array([[atom["x"], atom["y"], atom["z"]] for atom in polymer.atoms])
polymer_center = np.mean(polymer_coords, axis=0)
direction = port1_pos - polymer_center
direction = direction / np.linalg.norm(direction)

# Target position for port2: extend from port1 along direction
bond_length = 1.43  # C-O bond length
target_pos = port1_pos + direction * (bond_length * 2)

# Translation
translation = target_pos - port2_pos

# Calculate rotation to align port2 direction opposite to port1 direction
monomer2_center = np.mean(np.array([[atom["x"], atom["y"], atom["z"]] for atom in monomer2.atoms]), axis=0)
port2_direction = port2_pos - monomer2_center
port2_direction = port2_direction / np.linalg.norm(port2_direction)

# Calculate rotation axis to flip direction (180 degrees)
axis = np.cross(port2_direction, -direction)
axis_norm = np.linalg.norm(axis)
if axis_norm < 1e-6:
    # Vectors are parallel/anti-parallel, use perpendicular axis
    if abs(port2_direction[0]) < 0.9:
        axis = np.array([1, 0, 0])
    else:
        axis = np.array([0, 1, 0])
else:
    axis = axis / axis_norm

# Rotate 180 degrees around the calculated axis
angle = np.pi
monomer2.rotate(axis=axis.tolist(), angle=angle, about=monomer2_center.tolist())

# Apply translation to align ports
monomer2.move(delta=translation.tolist())

# Generate topology for monomer2
monomer2.get_topo(gen_angle=True, gen_dihe=True)

# Type monomer2
typifier.typify(monomer2)

# Merge into polymer
polymer.merge(monomer2)

# Regenerate topology for merged polymer
polymer.get_topo(gen_angle=True, gen_dihe=True)

# Re-type the entire polymer
typifier.typify(polymer)

print(f"✅ Step 2 completed:")
print(f"   Added monomer: {len(monomer2.atoms)} atoms")
print(f"   Polymer total: {len(polymer.atoms)} atoms")

# Export to LAMMPS
export_frame_to_lammps(polymer, output_dir / "step2_two_monomers.data", box_size=25.0)
print(f"   Exported: {output_dir / 'step2_two_monomers.data'}")

# Keep copies for step 3
left_monomer_copy = left_monomer.copy()
right_monomer_copy = monomer2.copy()

✅ Step 2 completed:
   Added monomer: 31 atoms
   Polymer total: 62 atoms
   Exported: case0_output/step2_two_monomers.data


## Step 6: Connect Monomers and Remove Water

Run the dehydration reaction to connect the two positioned monomers:
- Forms an ether bond between the monomers
- Removes water (H₂O) as a byproduct
- Regenerates topology and re-types the product

In [6]:
# Create reacter for dehydration
dehydration = Reacter(
    name="dehydration_ether",
    port_selector_left=select_carbon_from_oh,
    port_selector_right=select_port_atom,
    leaving_selector_left=select_hydroxyl_group,
    leaving_selector_right=select_h_from_oh,
    bond_former=form_single_bond,
)

# Run reaction on the two positioned monomers
result = dehydration.run(
    left=left_monomer_copy,
    right=right_monomer_copy,
    port_L=">",
    port_R="<",
    compute_topology=True,
)

product = result.product

# Regenerate topology for product
product.get_topo(gen_angle=True, gen_dihe=True)

# Re-type the product
typifier.typify(product)

print(f"✅ Step 3 completed:")
print(f"   Reacted polymer: {len(product.atoms)} atoms")
print(f"   Removed atoms: {len(result.removed_atoms)} atoms (water: O+H+H)")
print(f"   New bonds: {len(result.new_bonds)} bonds")

# Export to LAMMPS
export_frame_to_lammps(product, output_dir / "step3_connected.data", box_size=25.0)
print(f"   Exported: {output_dir / 'step3_connected.data'}")

✅ Step 3 completed:
   Reacted polymer: 59 atoms
   Removed atoms: 3 atoms (water: O+H+H)
   New bonds: 1 bonds
   Exported: case0_output/step3_connected.data


## Summary

This notebook demonstrated the complete step-by-step workflow for building and connecting polymer monomers:

1. ✅ Loaded OPLS-AA force field
2. ✅ Built first monomer from BigSMILES
3. ✅ Added second monomer with manual positioning (move/rotate)
4. ✅ Connected monomers via dehydration reaction
5. ✅ Exported each step to LAMMPS format

The generated `.data` files can be used for visualization and LAMMPS simulations:
- `step1_monomer.data`: First monomer
- `step2_two_monomers.data`: Two monomers positioned but not connected
- `step3_connected.data`: Connected polymer after reaction